### **notebook_plotter() by D. Principe (principe@space.mit.edu) -- MIT SDS April 2024**

#### This jupyter notebook is designed to demonstrate the potential for using Jupyter Widgets (ipywidgets) to interactively view sherpa models and data. This tool must be run with jupyter notebooks, sherpa and the ipywidgets package. This notebook and its associated function 'notebook_plotter()' will create an interactive plot window allowing users to modify model parameters in real-time to visually inspect how an unconvolved model would look convolved with the instrument associated with the sherpa dataset. This function has current features which include:

(a) inspecting sherpa or xspec models over a grid with a user-supplied spectral resolution (unlike plot_source() where the resolution is provided by the ARF file)

(b) interactively plot both the unconvolved model and the model convovled with the loaded instrument responses. This feature can be useful for proposal planning if a user want's to fake data to know if a certain spectral feature will be observable with Chandra (or any other instrument loaded in the sherpa session). 

(c) interactively plot a loaded dataset and the model convolved with responses. Users can alter parameters to see how that could affect their spectral fitting. This is particularly useful for identifying degeneracies between model parameters.

(d) create multiple interactive plots using multiple notebook cells and they remain (somewhat) independent.

#### This tool is not necessarily dependent on Chandra data. This tool should work in sherpa with other telescope spectra (e.g., XMM-Newton, NuSTAR) and their associated responses.

#### Current restrictions of the tool are:

- You can not plot more than one model type or dataset on the same interactive plot. Instead, you can use the notebook_plotter dataset_id parameter to switch between loaded datasets.

- the widget slider relies on a reasonable range of parameter values. If some model parameters have a minimum of 0 and maximum of 1E23 (for example) then a slider is not going to function very well over such a large range. Users will have to set the min/max values of parameters (eg., a1.nH.max = 100) with physically-reasonable values before calling the interactive model. 

#### More information about Jupyter Widgets (ipywidgets) can be found here:

https://ipywidgets.readthedocs.io/en/latest/index.html


### **USAGE INSTRUCTIONS**



This tool requires the ipywidget module which can be installed using either the pip ('pip install ipywidgets') or the conda install method ('conda install ipywidgets'). Once CIAO is initialized you should be able to run this notebook by typing "jupyter notebook FILENAME" where FILENAME is the location of this file.

This notebook includes the following three examples:

- EXAMPLE 1: Navigating the plotter and examining an unconvolved model.

- EXAMPLE 2: Loading and fitting a spectrum while visually inspecting how the model parameters may influence the overall fit.

- EXAMPLE 3 - Simulating a spectrum for a proposal with help from notebook_plotter().

The 'notebook_plotter()' function is described more below and only technically requires a model to be defined but other features also require a response (so it can plot the convolved model and counts data). 

If for some reason the notebook_plotter function does not properly plot, please make sure you 'trust' the jupyter notebook file when opening it in your browser.

In [ ]:
from sherpa_contrib.notebook_plotter import notebook_plotter

from sherpa.astro.ui import *
import numpy as np
import matplotlib.pyplot as plt    

#### We can use the help(notebook_plotter) to learn more about the function and its parameters.

In [ ]:
help(notebook_plotter)

#### This notebook includes several examples of how to use notebook_plotter(). As such, it's a good idea to run sherpa's clean() function in case you want to re-run this example after running later examples in this notebook.

In [ ]:
clean()

## **EXAMPLE 1: Navigating the plotter and examining an unconvolved model**

#### Let's use sherpa to generate a spectral model and plot it with notebook_plotter().The model here is an absorbed, one-temperature thermal plasma typical of coronal emission from a bright, young pre-main sequence star.

In [ ]:
test_model = xstbabs.ma1 * (xsapec.p1)
test_model #we can print the model parameters out to the notebook 

#### Looking over the model parameters (above), we'll want to constrain what will be plotted with more physically-appropriate limits. For example, xspec xstbabs model has a maxmimum nH value of 1,000,000 (1E28 atoms/cm2) and that value is not relevant for nearby young stars. 

In [ ]:
ma1.nH.min = 0.01
ma1.nH.max = 40
p1.redshift.min = 0

#### Now let's open this model in the notebook_plotter() by simply calling it as a variable to the notebook_plotter function.

In [ ]:
notebook_plotter(test_model)

#### *Here you can see the basic notebook interface with the model parameters on the left and the plotting options on the right. Each parameter of the tool is controlled using an ipywidget and there are several different ipywidgets to interact with. You can activate 'sliding widgets' in one of three ways: (a) by clicking on the blue circle and dragging the parameter left or right, (b) clicking on the line where you want the slider to go, or (c) clicking the value to the right of the widget and typing in a new value. 'Float text' widgets are boxes with a value displayed. You can click in the box and enter your own value or you can hover to the right of the box and click up or down to change the parameter by one widget unit. The 'dropdown' widgets can be selected by clicking them and picking an option from the dropdown menu. Note that you can interactively change the size of the plotting window using the figsize_x and y parameter sliders.*

#### *This plotting tool can also be opened in a specific state by calling the function with the parameters listed in the plotting options using the same syntax. If the parameter requires a string and the wrong syntax is used then the plot warns you that the tool's default value for that parameter will instead be used.*

In [ ]:
notebook_plotter(test_model, log_axis = 'ylog', xlim_low = 0.3, xlim_high =10, figsize_y = 3, plot_type = 'model_unconvolved')

#### Something to note is that the test_model paramater values **WILL** be updated in your sherpa session with the widget values selected in the interactive plot after running notebook_plotter(). This can be useful when trying to fit models to data but dangerous if you are not aware that model values are being updated. 

In [ ]:
test_model()

#### *Here we can see that the sherpa model values have been updated to the same as those shown in the interactive plot.*

#### *When running notebook_plotter() in some browsers, adjusting parameters with a mouse automatically scrolls the entire plot window every time a parameter is updated. A solution to this behavior is to make sure there are several cells (empty or full) below the cell in your jupyter notebook where you call notebook plotter.*

## **EXAMPLE 2: Loading and fitting a spectrum while visually inspecting how the model parameters may influence the overall fit.**

#### This example is meant to show how to use notebook_plotter() with a real dataset. However, that would require downloading a dataset and extracting a spectrum which is beyond the scope of this guide. Instead, we will simulate a spectrum and save it as a dataset.  

#### Users who wish to learn more are about extracting spectra for real datasets are encouraged to see the following CIAO thread (https://cxc.cfa.harvard.edu/ciao/threads/pointlike/). 

#### In order to simulate a spectrum we will first need to download Chandra ARF and RMF responses for our instrument of choice (ACIS-S). For this example, we will use recent responses generated for proposal planning purposes only can be found here (https://cxc.cfa.harvard.edu/caldb/prop_plan/imaging/). Please note that these responses are only meant to be used for simulating spectra and should not be used to analyze real Chandra observations.

In [ ]:
from urllib.request import urlretrieve

#download the ACIS-S ARF
url_arf = ('https://cxc.cfa.harvard.edu/caldb/prop_plan/imaging/CY26/aciss_aimpt_cy26.arf')
filename_arf = 'aciss_aimpt_cy26.arf'
urlretrieve(url_arf, filename_arf)

#download the ACIS-S RMF
url_rmf = ('https://cxc.cfa.harvard.edu/caldb/prop_plan/imaging/CY26/aciss_aimpt_cy26.rmf')
filename_rmf = 'aciss_aimpt_cy26.rmf'
urlretrieve(url_rmf, filename_rmf)


#### Now we can define a model, set some parameter values and simulate a spectrum using fake_pha(). The model here is an absorbed, one-temperature thermal plasma typical of coronal emission from a bright, young pre-main sequence star.

In [ ]:
clean()

#lets define a model and set it as the source_model
model_for_simulation = xstbabs.asim * (xsvapec.psim)
set_source(model_for_simulation)

#here we can set some model parameters and the simulation exposure time
asim.nH = 0.1 
psim.kT = 2 #keV
psim.norm  = 1E-3

simulation_exposure = 50E3 #ks

#simulate the spectrum and save it as a file in order to replicate the experience of loading in a real sources spectrum.
fake_pha(1, arf = filename_arf, rmf = filename_rmf, exposure = simulation_exposure)
save_pha(1,'simulated_spectrum.pi', clobber = 'yes')

model_for_simulation #lets print the model parameters to the screen

#### Now that we have a (simualted) dataset, lets load it in to sherpa as though it is a real source's spectrum. Note, the ARF and RMF should be automatically identified with the load_pha() command as those were the responses used to generate the spectrum.

In [ ]:

clean() #here we run clean() so the simulated dataset and source model are removed from the sherpa session.

pha_file = 'simulated_spectrum.pi'

load_pha(pha_file)

plot_data()

#### We can group the spectrum and plot it using the plot_data() sherpa command.

In [ ]:
#group the spectrum and ignore everthing but 0.5-8.0 keV counts
group_counts(1,10)
notice()
ignore(0,0.5)
ignore(8,)

#replot the grouped spectra with log yscale
plot_data()
plt.yscale('log')

#### Now that we have some data loaded, we will want to define a spectral model before we fit the data.

In [ ]:
#lets define a model and set it as the source_model
test_model_2 = xstbabs.ma2 * (xsvapec.p2)
set_source(test_model_2)
test_model_2


#### Since we are using data simulated with a model, we happen to know the exact model parameter values. This would not be the case for a real observation. That being said, we can still use notebook_plotter() to set some reasonable parameter values before running fit(). Try modifying the parameters ma2.nH, p2.kT and p2.norm until the model looks more like the dataset. Instead of using the sliders, you can also click the values to the right of the sliders and set them with keyboard input.

In [ ]:
ma2.nH.max = 30 #lets set nH to a physically-reasonable nH for nearby young stars

notebook_plotter(test_model_2, log_axis = 'ylog', autoscale = 'y', figsize_y = 3, plot_type = 'data_and_convolved_model')

#### Note, that the actual values used to simulate the dataset are ma2.nH = 0.1 atoms/cm2 , p2.kT = 2.0 keV , and p2.norm = 1E-3. Also note that sherpa includes the guess() function for estimating parameter ranges given a loaded dataset.

#### Now lets fit() the data and overplot the best-fit model.

In [ ]:
#lets fit the model to the loaded dataset and inspect the data+fit
fit()

plot_fit()
plot_model(overplot=True)

plt.yscale('log')

#### Now let's re-run notebook_plotter(). We will see that the model parameters in sherpa have been set to the values determined in the fit. Note that previous jupyter notebook cells running notebook_plotter() will **not** be updated with these best-fit parameter values. It is not recommend to run notebook_plotter() in multiple cells if you intend to be careful about setting model parameters (like you would right before a fit).

In [ ]:
notebook_plotter(test_model_2, log_axis = 'ylog', autoscale = 'y', figsize_y = 3, plot_type = 'data_and_convolved_model')

#### We can also use the notebook_plotter to view the contribution of the model (purple) to the convolved spectrum (orange). Try adjusting the parameters to see how they affect the spectrum. While we can access this plot using the previous cell's plot by selecting 'both_models' from the plot_type window, it makes more sense for a tutorial to replot as we do in the next cell.

#### NOTE: Adjusting model parameters in this cell will update the source model parameters in sherpa.


In [ ]:
notebook_plotter(test_model_2, log_axis = 'ylog', figsize_y = 3, plot_type = 'both_models', autoscale = 'none', ylim_low = '1E-8', ylim_high = '1')

#### Note, when using plot_type = 'both_models', the plotting options such as autoscale, xy-limits and log_axis refer only to the unconvolved model (purple).  The y-axis of the convolved model (orange), which is plotting different y-axis units, will always autoscale to fit within the ymin and ymax ranges set by the notebook_plotter. Unlike the unconvolved model, the convolved model is affected by the response (e.g., the ARF and RMF) of the telescope.

#### You can also view this dataset in wavelength space with the set_analysis() command. Keep in mind this will set the analysis to 'wavelength' for the entire sherpa session or untill you switch back using set_analysis('energy').

In [ ]:
#We can also see what this spectra looks like in wavelength space by using set_analysis()
set_analysis('wavelength')

notebook_plotter(test_model_2, log_axis = 'ylog', figsize_y = 3, plot_type = 'both_models', autoscale = 'y', ylim_low = 1E-15, ylim_high = 10, xlim_low = 0, xlim_high = 20)

## **EXAMPLE 3 - Simulating a spectrum for a proposal with help from notebook_plotter()**



#### Let's say we're interested in identifying a sample of young, pre-main sequence stars whose spectrum may include a 6.4 keV emission feature characteristic of neutral iron fluoresence emission from cold circumstellar disk gas irradiated by coronal X-rays. It would help to first look at spectral models typically used to fit young stellar X-ray spectra to determine what characteristics of the spectrum are important for detecting an emission line at 6.4 keV.




#### We're going to use sherpa's fake_pha() command to simulate a spectrum but first we will need to choose a model. Let's go with a simple plasma model typically used to fit spectra of young stars. For the moment, we will choose a model that does **not** include the emission line of interest at 6.4 keV because most stars don't show this feature. We will add that in later. In this example we will end up working with two sherpa datasets so we will explicitly set the dataset id parameters when calling the relevant sherpa functions.


In [ ]:
clean() #lets run clean() to remove the datasets of previous examples from our sherpa session.
test_model_3 = xstbabs.ma3 * (xsapec.p3)

#lets set some parameters typical of young stellar coronae
ma3.nH = 0.1
p3.kT = 1.3
p3.Abundanc = 0.5
p3.norm = 3E-3
test_model_3

#lets display the model parameters that were set
test_model_3

#### To fake a spectrum for a future proposal, we will need to use chandra responses (ARF AND RMF) relevant for the year of our proposed observations and for our instrument of choice. For this case, we will use the same responses in example 2 which are for Cycle 26 and can be found here: https://cxc.cfa.harvard.edu/caldb/prop_plan/imaging/. Please note that these responses are only meant to be used for simulating spectra and should not be used to analyze real Chandra observations.

In [ ]:
from urllib.request import urlretrieve

#download the ACIS-S ARF
url_arf = ('https://cxc.cfa.harvard.edu/caldb/prop_plan/imaging/CY26/aciss_aimpt_cy26.arf')
filename_arf = 'aciss_aimpt_cy26.arf'
urlretrieve(url_arf, filename_arf)

#download the ACIS-S RMF
url_rmf = ('https://cxc.cfa.harvard.edu/caldb/prop_plan/imaging/CY26/aciss_aimpt_cy26.rmf')
filename_rmf = 'aciss_aimpt_cy26.rmf'
urlretrieve(url_rmf, filename_rmf)


#### Now lets use fake_pha() to simulate a typical young stellar coronal source in a 100 ks observation. We will explicitly assign dataset id to 1 because later we will work with another dataset id.

In [ ]:
set_source(1, test_model_3)

exposure_time = 100E3 # 100 ks

fake_pha(1, arf=filename_arf, rmf=filename_rmf, exposure = exposure_time)

#### We can group the spectrum and ignore everthing but 0.5-8.0 keV counts and display the faked data with the convolved model using the sherpa command plot_model()

In [ ]:
group_counts(1,7)
notice()
ignore(0,0.5)
ignore(8,)
plot_data(1)
plt.yscale('log')
plot_model(1,overplot=True)

#### Lets notebook_plotter() to inspect the simulated (faked) spectrum and its underlying model.

In [ ]:
ma3.nH.min = 0.01
ma3.nH.max = 30
p3.kT.min = 0.2
p3.kT.max = 9

notebook_plotter(test_model_3, dataset_id = 1, log_axis = 'ylog', autoscale = 'none',  plot_type = 'data_and_convolved_model', xlim_low = 0.3, xlim_high = 8.0, ylim_low = 1E-7, ylim_high = 1)

#### *We can see that there are not many counts in the spectrum >~ 5 keV which is going to make it difficult to detect any specific emission features in this part of the spectrum. What happens if you take the p3.kT widget slider and increase the plasma temperature? In the unconvolved (purple) model you'll notice increased emission at high energies (>~ 2 keV) relative to emission at soft energies (<~ 2keV). In other words, the spectrum is getting harder. You'll see the same effect if you note the convolved model (orange) y-axis values on the right side of the plot. The counts/s/keV produced by the model increases at high energies (> 2 keV) relative to low energies as you increase plasma temperature (kT). Lets take a closer look at the underlying plasma model in the next cell.*


In [ ]:
p3.kT = 1.3 #just in case you changed the value of p3.kT in the interactive widget, lets set it back to 1.3 keV for the purposes of this notebook.

notebook_plotter(test_model_3, dataset_id =1, log_axis = 'ylog', autoscale = 'none',  plot_type = 'both_models', xlim_low = 0.3, xlim_high = 8.0, ylim_low = 1E-7, ylim_high = 1)

#### *Here we see the underlying plasma model (purple) used to simulate the spectrum and the same model convolved with the chandra ACIS-S responses (orange). For emission line sources like young, pre-main sequence stars, the shape of the spectrum in low resolution spectra (i.e., CCD imaging-resolution) is driven in large part by all the underlying emission lines.*

#### *Try changing the plasma temperature parameter in this interactive model and note how the shape of the convolved model changes. As you increase plasma temperature, emission lines at higher energies increase in strength relative to those at energies <~ 1 keV pushing up the high enery portion of the spectrum. Of note, is the very bright series of lines at 6.7 keV assocaited with with iron (Fe XXV). This 6.7 keV feature has been identified in the spectra of many hot stars and is typically used to demonstrate that the underyling plasma is indeed hot. You can see how if you reduce the plasma temperature, this 6.7 keV emission feature is reduced.*

#### Now, lets take a look at this simulated spectrum but with a model that includes a 6.4 keV gaussian line meant to represent Fe flourescence emission. We will set all the other parameters initially to the same values as our simulated spectrum but this time adding in a 6.4 keV gaussian component. Since we want to see what the new model looks like convolved with the ACIS response, we will have to explicitly set the new model to dataset 1 using set_source().

In [ ]:
#now, lets take at this simulated spectrum but with a model that includes a 6.4 gaussian line meant to represent Fe flourescent emission. We will set all the other parameters initially to the same values as are simulated spectrum but this time add in the gaussian component.

#You can now see the presence of a gaussian line at 6.4 keV in the unconvolved model (purple) and its contribution to the convolved model (orange). Try increasing the g4.norm parameter to see how an increase in line flux affects the spectrum. Given the current model parameters, it appears you'll need a g4.norm value >~ 1E-8 in order to identify any feature. 

test_model_4 = xstbabs.ma4 * (xsapec.p4 + xsgaussian.g4)

ma4.nH = 0.1
p4.kT = 1.3
#p4.kT = 3.0
p4.Abundanc = 0.5
p4.norm = 3E-3
g4.sigma = 0.0
g4.norm = 5E-7
g4.LineE = 6.4

ma4.nH.min = 0.01
ma4.nH.max = 30
p4.kT.min = 0.3
p4.kT.max = 5

g4.LineE.min = 6.39
g4.LineE.max = 6.41
g4.norm.max = 1E-2
g4.sigma.max = 0.05

set_source(1, test_model_4) 

notebook_plotter(test_model_4, dataset_id=1, log_axis = 'ylog', autoscale = 'none',  plot_type = 'both_models', xlim_low = 0.3, xlim_high = 8.0, ylim_low = 1E-7, ylim_high = 1)


#### *You can now see the presence of a gaussian line at 6.4 keV in the unconvolved model (purple) and its contribution to the convolved model (orange). Try increasing the g4.norm parameter by entering a higher value in the widget box to see how an increase in line flux affects the spectrum. Given the current model parameters, it appears you'll need a g4.norm value >~ 1E-8 in order to identify any feature. If you wish to zoom-in on this line, you can set the xlim_low and xlim_high values in the plotting options.*

#### Now lets use the plot_type 'data_and_convolved_model' option to look at our new convolved model overlaid on the previous faked spectrum which does not include a 6.4 keV component.

In [ ]:
#here we will reset the model values in case they were modified too drastically in the previous notebook_plotter() cell.
ma4.nH = 0.1
p4.kT = 1.3
p4.Abundanc = 0.5
p4.norm = 3E-3
g4.sigma = 0.0
g4.norm = 5E-7
g4.LineE = 6.4

notebook_plotter(test_model_4, dataset_id =1, log_axis = 'ylog', autoscale = 'none',  plot_type = 'data_and_convolved_model', xlim_low = 0.3, xlim_high = 8.0, ylim_low = 1E-7, ylim_high = 1)

#### *Here we can see the 6.4 emission line component in the convolved model but there are relatively few counts in our faked spectrum >~ 5 keV which will make it difficult to detect without significantly more exposure time. We saw in a previous cell that increasing the plasma temperature will increase the number of counts at higher energies for a set normalization. Try increasing the value of p4.kT to be 3.0 keV. Compared to the previous simulation, this new, (hotter) model has a lot more flux > 5 keV although we now have to increase the value of g4.norm to ~3E-6 so it is visible above the continuum of the spectrum.*

#### *Since this science case is focused on detecting a 6.4 keV feature, it will be easier to detect such a feature with Chandra by focusing on young stars that already exhibit hot plasma (e.g., >~ 3 keV) as there will be more counts in the region where we expect this feature.*

#### So now let's try simulating another spectrum with this gaussian component at 6.4 keV and the above model parameters. While you should be able to run fake_pha() using the model values set in the previous cell's notebook_plotter(), its safer to just reset the values here before simulating. We will assign this new spectrum to a new dataset 2.

In [ ]:
#so now lets try simulating a spectrum with this gaussian component at 6.4 keV and the above model parameters. While you should be able to run fake_pha() using the model values set in the previous cell's notebook_plotter(), its safer to just reset the values here before simulating.

ma4.nH = 0.1
p4.kT = 3.0
p4.Abundanc = 0.5
p4.norm = 3E-3
g4.sigma = 0.0
g4.norm = 3E-6
g4.LineE = 6.4

set_source(2, test_model_4)

exposure_time = 100E3 #ks

fake_pha(2, arf=filename_arf, rmf=filename_rmf, exposure = exposure_time)

#### Like before, lets group the spectrum and ignore everthing but 0.5-8.0 keV and display the faked data with the convolved model using the sherpa plot_data() and plot_model() commands.

In [ ]:
#We can group the spectrum and ignore everthing but 0.5-8.0 keV and display the faked data with the convolved model using sherpa commands
group_counts(2,10)
notice()
ignore(0,0.5)
ignore(8,)
plot_data(2)
plt.yscale('log')
plot_model(2,overplot=True)

#### In this simulated spectrum, we can now identify both the 6.4 and 6.7 keV emission lines.  Let's open it in notebook_plotter() for a final view.

In [ ]:
notebook_plotter(test_model_4, dataset_id=2, log_axis = 'ylog', autoscale = 'none',  plot_type = 'data_and_convolved_model', xlim_low = 0.3, xlim_high = 8.0, ylim_low = 1E-7, ylim_high = 1)

#### Now we can compare dataset 2 and its associated test_model_4 to our previous dataset 1 and its original model test_model_3. In a previous cell, we used set_source(1, test_model_4) to demonstrate what our 6.4 keV model would look like on the first simulated spectrum. If we want to use notebook_plotter to see dataset 1 with test_model_3 then we first need to re-assign test_model_3 to dataset 1 using set_source(1, test_model_3). NOTE: If you want to plot convolved models or data using notebook_plotter then the model variable (in this case its test_model_3) must be set with set_source() to the same dataset_id value called in notebook_plotter. This ensures the proper responses are used when convolving data.

In [ ]:
set_source(1, test_model_3)

notebook_plotter(test_model_3, dataset_id=1, log_axis = 'ylog', autoscale = 'none',  plot_type = 'data_and_convolved_model', xlim_low = 0.3, xlim_high = 8.0, ylim_low = 1E-7, ylim_high = 1)

#### Now that we have a simulation of a source where we can detect these lines, a next and final step (left to the reader) would be to explore the parameter space for this model and simulation. If proposing to observe a real source, you'll want to simulate spectra using values appropraite for the real object (e.g., flux, plasma temperature, median energy, nH). After modifying the relevant parameters in the model, you can re-simulate a spectrum for each source to determine the best instrument setup and exposure time request. The 6.4 keV emission feature discussed in this example is rare and is limited to young stars hosting circumstellar disks with reservoirs of neutral Fe. 

#### A helpful tool in determing what normalization parameter you can use for your simualtion is the sherpa command calc_energy_flux(). This allows you to evaluate your loaded model with an energy range to determine the flux. If you'd like to simulate a real source's spectrum, be sure to match the simulation's flux with your source of interest's flux (via the plasma normalization in this case).